In [125]:
import numpy as np
from collections import Counter
import math
import pandas as pd

### Question No 1

In [126]:
test_dict = {"<10": 0.08, "10-15": 0.27, "15-20": 0.10,
             "20-25":0.11,"25-30":0.15,"30-35":0.20,"35-37":0.07,
             "37+":0.02}

In [127]:
def truckSim(n,d):
    truck_levels = [] # Initialize an empty list
    for j in range(n): # Run this loop for n trucks
        cumulative_prob = 0 # Initialize a cumulative probability
        random_num = np.random.uniform(0,1) # Generate a number between 0 and 1
        for (k,v) in d.items(): # Run this loop for each value in the dictionary
            cumulative_prob += v # Add the probability of this particular dictionary entry
            if cumulative_prob > random_num: # Once the cumulative probability is higher than the random number...
                truck_levels.append(k) # ...add the key to the list
                break # and break the for loop
    return truck_levels

### Question No 2

In [134]:
simlist = truckSim(1000,test_dict)

In [181]:
def truck1CI(alpha,keyname,n,simlist):
    finallist = []
    deltalist = []
    for i in range(n):
        templist = np.random.choice(simlist,len(simlist))
        pos = Counter(templist)[keyname]
        finallist.append(pos/len(templist))
    samplemean = np.mean(finallist)
    stderror = np.std(finallist)/np.sqrt(n)
    for i in range(n):
        bs =np.random.choice(finallist,len(simlist))
        deltalist.append(np.mean(bs)-samplemean)
    deltalist = sorted(deltalist,reverse=False)
    ub = deltalist[int(alpha*100)]
    lb = deltalist[int(1-alpha*100)]
    return pd.Series([samplemean,samplemean+lb,samplemean+ub,stderror],index = ['Est.Value','CI Lower Bound',
                                                                                'CI Upper Bound','Standard Error'])

In [182]:
df = pd.DataFrame([[1-0.1, '30-35', 100],[1-0.1, '30-35', 1000],[1-0.1, '30-35', 2500],
                   [1-0.1, '30-35', 5000],[1-0.1, '<10', 100],[1-0.1, '<10', 1000],
                   [1-0.1, '<10', 2500],[1-0.1, '<10', 5000]],
                  columns=['Confid.Level','Truck Fill Level','No of Bootstraps'])
df['Est.Value'],df['CI Lower Bound'],df['CI Upper Bound']= np.zeros(8),np.zeros(8),np.zeros(8)
df2= df.apply(lambda x:(truck1CI(x[0],x[1],x[2],simlist)),axis=1)
df['Est.Value'] = df2['Est.Value']
df['CI Lower Bound']= df2['CI Lower Bound']
df['CI Upper Bound']= df2['CI Upper Bound']
df['Standard Error']= df2['Standard Error']

In [183]:
df

,Confid.Level,Truck Fill Level,No of Bootstraps,Est.Value,CI Lower Bound,CI Upper Bound,Standard Error
0,0.9,30-35,100,0.197370,0.197068,0.197826,0.001094
1,0.9,30-35,1000,0.197312,0.197836,0.196846,0.000378
2,0.9,30-35,2500,0.198010,0.198734,0.197278,0.000250
3,0.9,30-35,5000,0.197797,0.198655,0.196950,0.000178
4,0.9,<10,100,0.085860,0.085419,0.086272,0.000944
5,0.9,<10,1000,0.086214,0.086634,0.085805,0.000285
6,0.9,<10,2500,0.085967,0.086480,0.085467,0.000175
7,0.9,<10,5000,0.086175,0.086752,0.085608,0.000125
